In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from MCForecastTools import MCSimulation
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

def get_stock_data(tickers, start_date="2020-01-01", end_date=datetime.now().strftime('%Y-%m-%d')):
    stock_data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]
    return stock_data

# Prompt the user for tickers
user_input = input("Please enter stock tickers separated by commas (e.g. AAPL,MSFT,GOOGL): ")
tickers = [x.strip().upper() for x in user_input.split(",")]

# Prompt the user for the weights of each ticker
print(f"Please provide weights for {', '.join(tickers)} respectively, separated by commas (e.g. 0.4,0.3,0.3): ")
weights_input = input().split(',')
weights = [float(x.strip()) for x in weights_input]

# Validate the weights sum to 1
while not (0.99 <= sum(weights) <= 1.01):  # Tolerating small rounding errors
    print("Weights do not sum up to 1. Please try again.")
    print(f"Please provide weights for {', '.join(tickers)} respectively, separated by commas: ")
    weights_input = input().split(',')
    weights = [float(x.strip()) for x in weights_input]

# Prompt the user to choose the simulation length
user_input_years = input("Please enter the number of years for the simulation (3, 4, or 5): ")
while user_input_years not in ["3", "4", "5"]:
    print("Invalid input. Please enter 3, 4, or 5.")
    user_input_years = input("Please enter the number of years for the simulation (3, 4, or 5): ")
S
num_years = int(user_input_years)

# Fetch the stock data using the function
portfolio_data = get_stock_data(tickers)

# Structure portfolio_data to have multi-level columns
portfolio_data.columns = pd.MultiIndex.from_product([portfolio_data.columns, ['close']])

# Calculate daily returns and add to the portfolio data
for ticker in tickers:
    portfolio_data[(ticker, 'daily_return')] = portfolio_data[ticker]['close'].pct_change()

# Display a portion of the data to verify structure
print(portfolio_data.head())

# Run the Monte Carlo Simulation
mc_sim = MCSimulation(portfolio_data=portfolio_data, weights=weights, num_simulation=500, num_trading_days=252*num_years)
mc_sim.calc_cumulative_return()
mc_sim.plot_simulation()
mc_sim.plot_distribution()

# Compute summary statistics from the simulated daily returns
simulated_returns_data = {
    "mean": list(mc_sim.simulated_return.mean(axis=1)),
    "median": list(mc_sim.simulated_return.median(axis=1)),
    "min": list(mc_sim.simulated_return.min(axis=1)),
    "max": list(mc_sim.simulated_return.max(axis=1))
}

# Create a DataFrame with the summary statistics
df_simulated_returns = pd.DataFrame(simulated_returns_data)

# Display sample data
print(df_simulated_returns.head())

# Calculate portfolio performance over time under `initial_investment`
initial_investment = 1000
cumulative_pnl = initial_investment * df_simulated_returns['mean']

# Plot the cumulative returns over time
cumulative_pnl.plot(figsize=(10,5))
plt.title(f"Cumulative Portfolio Performance Over {num_years} Years")
plt.show()




Please provide weights for  respectively, separated by commas (e.g. 0.4,0.3,0.3): 


ValueError: could not convert string to float: ''